In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wh_file_path='/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv'
world_happiness = pd.read_csv(wh_file_path)
world_happiness.rename(columns=lambda x: x.strip().lower().replace(' ','_'),inplace=True)
world_happiness.head()

After this terrible pandemic, I really can't decide where I want to travel to next. There are a plethora of options in the world where I can get a sense of culture, try new foods, and meet new people. I think that the level of happiness that people are in the country will help me zone in on where I would like to go next.

**Priorities**
As a woman travelling alone, I want to be able to depend on kind people around me in case I am in trouble. I also do not want to feel like I have to give up any of my womanly rights to do as I please or walk alone. 

Let's explore the data to find any factors that may help my priorities!

In [ ]:
#check if item in any column is null
world_happiness.isnull().any()

In [ ]:
#check if there are same number of rows in each column
#understand general trend of data
world_happiness.describe()

**Factors**

Factors that will represent kind supportive people are social support and generosity.Another factor is freedom to make life choices, which represents women's rights and choices. 

Each factor adds different amount of weight to the country's overall happiness ladder scores. Since I take priority in social support, generosity and freedom to make life choices, then I want to know which countries take high importance with the respective factors.

In [ ]:
world_happiness_filter = world_happiness[(world_happiness["social_support"]>=0.80) & (world_happiness["generosity"]>=0.15) & (world_happiness["freedom_to_make_life_choices"]>=0.75)]
world_happiness_filter

With 4 regions and 9 countries meeting my criteria, I want to find out where the happiest people in the regions I will visit.

In [ ]:
world_happiness_group = world_happiness_filter.groupby(["regional_indicator","country_name"]).agg(ladder_score=pd.NamedAgg(column="ladder_score",aggfunc='max'),std_err_ladder_score = pd.NamedAgg(column="standard_error_of_ladder_score",aggfunc='max'))
world_happiness_group

I could look over the chart to know which country has the highest ladder score in each country. However, I want to automatically store those countries and their ladder scores to later graph them. I must prepare the filtered data to be more easily iterable 

In [ ]:
#distinct list of regions to look for and find highest ladder score
RegIndicatorDistinctArray = list(set(np.array(world_happiness_filter["regional_indicator"])))
world_happiness_filter=world_happiness_filter.reset_index(drop=True)
world_happiness_filter

In [ ]:
j = 0
maxhappycountries = []
#run through each region once 
while j < len(RegIndicatorDistinctArray):
    indices=[]
    maxhappy = 0
    maxhappycountry = '  '
    
    #for each region, run through regional indicator column to gain all the indices for where the region lies
    for index in range(0,len(world_happiness_filter)): 
        if world_happiness_filter.at[index,"regional_indicator"] == RegIndicatorDistinctArray[j]:
            indices.append(index)
            
    #get the highest ladder score and the country name within the region 
    for i in indices:
        if world_happiness_filter.at[i,"ladder_score"] > maxhappy:
            maxhappy = max(maxhappy,world_happiness_filter.at[i,"ladder_score"])
            maxhappycountry = world_happiness_filter.at[i,"country_name"]
    maxhappycountries.append({"Country": maxhappycountry, "Maximum Happiness Score": maxhappy})
    j+=1
maxhappycountriesdf = pd.DataFrame(maxhappycountries)
print(maxhappycountriesdf)

In [ ]:
df = pd.DataFrame(data=maxhappycountries)
df
plt.figure(1)
df.plot.bar(x="Country",y="Maximum Happiness Score",title = "Maximum Happiness Scores per Country")

Iceland has the highest happiness score and lies within Western Europe. I would like to see if I could travel there and do a road trip in that region but I will only go if Western Europe's average happiness score is at least top three amongst the rest of the other countries.

In [ ]:
region_ladder_mean = world_happiness_filter.groupby(["regional_indicator"]).ladder_score.mean()
region_ladder_mean
rlmp = region_ladder_mean.plot.bar(x ="Region", y ="Average Happiness Score", title = "Average Happiness Score per Region")

Overall, Western Europe is the highest happiness region and fits all factors (social support, generosity, and freedom to make life choices). I cannot wait to travel to Iceland, the Netherlands, and the United Kingdom! All fairly different cultures and places I feel safe to travel alone to. 

Thanks for following my adventure!